In [ ]:
!nvidia-smi

Fri May 12 06:08:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q --upgrade transformers diffusers ftfy huggingface_hub
!pip install -q accelerate 
!pip install -q einops
!pip install -q gradio
!pip install -q open_clip_torch
!pip install -q --upgrade omegaconf
!pip install -q pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 12.1 MB/s eta 0:00

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
import os
from PIL import Image
import numpy as np
import albumentations as A
import albumentations.pytorch as AP
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

from base64 import b64encode
import numpy as np
import json
import cv2
# from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
import subprocess

# For video display:
from IPython.display import HTML
from pathlib import Path
from torch import autocast
from torchvision import transforms as tfms
import pytorch_lightning as pl
from transformers import logging #CLIPTextModel, CLIPTokenizer, 
from transformers import BlipProcessor, BlipForConditionalGeneration

# torch.manual_seed(1)

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

from huggingface_hub import hf_hub_download
from diffusers import DiffusionPipeline
from huggingface_hub import notebook_login

In [ ]:
if not (Path.home()/'.huggingface'/'token').exists(): notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Copying the code from github to colab
os.chdir('/content/')
!git clone https://github.com/anilbhatt1/ControlNet.git 
%cd '/content/ControlNet'
! ls

##Preparing dataloader 

In [ ]:
os.chdir('/content/')
!cp -r '/content/gdrive/MyDrive/EVA8_P1_S15' '.'
!pwd

In [ ]:
os.chdir('/content')
!pwd
!unzip -q '/content/EVA8_P1_S15/fill50k.zip'

In [ ]:
!ls '/content/fill50k/source' | wc -l
!ls '/content/fill50k/target' | wc -l

In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open('/content/fill50k/prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))
        self.width = 512
        self.height = 512

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']                
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread('/content/fill50k/' + source_filename)
        target = cv2.imread('/content/fill50k/' + target_filename)

        source = cv2.resize(source, (self.width, self.height), interpolation=cv2.INTER_NEAREST)
        target = cv2.resize(target, (self.width, self.height), interpolation=cv2.INTER_NEAREST)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [ ]:
dataset = MyDataset()
print(len(dataset))

item = dataset[234]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

##Training Fill50K from scratch

In [ ]:
os.chdir('/content/ControlNet')

In [ ]:
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict

# resume_path = '/content/models/control_sd15_ini.ckpt'
# resume_path = '/content/gdrive/MyDrive/EVA8_P1_S15_CTLNET_Wt/epoch=1-step=1894.ckpt'
batch_size = 8
logger_freq = 620
learning_rate = 1e-5
sd_locked = True
only_mid_control = False

# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml').cpu()
# model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

In [ ]:
# Misc
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=12, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(precision=32, callbacks=[logger], max_epochs=1)

# Train!
trainer.fit(model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('global_step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.95it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.96it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.96it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.96it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.96it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.96it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.9

In [ ]:
!cp -r '/content/ControlNet/image_log' '/content/gdrive/MyDrive/EVA8_S15_May12'